In [1]:
from pathlib import Path
from tqdm import tqdm
import csv
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import warnings
from natsort import natsorted
import pandas as pd

warnings.filterwarnings(action='ignore')

In [2]:
def prepare_RAVDESS_DS(path_audios):
    wav_paths, emotions, actors = [], [], []
    for path in tqdm(Path(path_audios).glob("*/*.wav")):
        name = str(path).split('/')[-1].split('.')[0]
        label = int(name.split("-")[2]) - 1  # Start emotions in 0
        actor = int(name.split("-")[-1])

        try:
            wav_paths.append(path)
            emotions.append(label)
            actors.append(actor)
        except Exception as e:
            # print(str(path), e)
            pass
        
    return wav_paths, emotions, actors

In [4]:
def save_melspectrogram(paths, emotions, actors):
    data = []
    for i, path in tqdm(enumerate(paths), desc='melspectrogram image generate.....'):
        y, sr = librosa.load(path, sr=16000)
        
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, win_length=512, window='hamming', hop_length=256, n_mels=256, fmax=sr/2)
        melspectrogram = librosa.power_to_db(S, ref=np.max)
        
        temp = np.zeros((256, 307))
        temp[:,:melspectrogram.shape[1]] = melspectrogram
        
        data.append({
            'melspectrogram_feature': temp,
            'emotion': emotions[i],
            'actor': actors[i]
        })
        
    df = pd.DataFrame(data)
    return df

In [5]:
df = save_melspectrogram(paths, emotions, actors)

melspectrogram image generate.....: 1440it [01:02, 23.01it/s]


In [6]:
from torchaudio.functional import compute_deltas
import torch

def get_deltas(feature):
    f = np.expand_dims(feature, 1)
    f = torch.Tensor(f)
    
    delta = compute_deltas(f)
    delta2 = compute_deltas(delta)
    
    ret = torch.cat([f, delta, delta2], dim=1)
    
    return ret

In [10]:
def generate_train_test(fold, df, save_path=""):
    """
    Divide the data in train and test in a subject-wise 5-CV way. The division is generated before running the training
    of each fold.
    :param fold:[int] Fold to create the train and test sets [ranging from 0 - 4]
    :param df:[DataFrame] Dataframe with the complete list of files generated
    :param save_path:[str] Path to save the train.csv and test.csv per fold
    """
    
    actors_per_fold = {
        0: [2,5,14,15,16],
        1: [3, 6, 7, 13, 18],
        2: [10, 11, 12, 19, 20],
        3: [8, 17, 21, 23, 24],
        4: [1, 4, 9, 22],
    }

    test_df = df.loc[df['actor'].isin(actors_per_fold[fold])]
    train_df = df.loc[~df['actor'].isin(actors_per_fold[fold])]

    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)
    
    X_train = np.array([data for data in train_df['melspectrogram_feature']])
    y_train = np.array([data for data in train_df['emotion']])
    X_test = np.array([data for data in test_df['melspectrogram_feature']])
    y_test = np.array([data for data in test_df['emotion']])
    
    X_train = get_deltas(features_train)
    X_test = get_deltas(features_test)
    
    with open(save_path+'.npy', 'wb') as f:
        np.save(f, X_train)
        np.save(f, y_train)
        np.save(f, X_test)
        np.save(f, y_test)
        
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [11]:
import time

for fold in range(5):
    
    save_path = os.path.join('5-CV', "fold"+str(fold))
    os.makedirs(save_path, exist_ok=True)
    
    generate_train_test(fold, df, save_path)
    time.sleep(10)

ValueError: operands could not be broadcast together with shapes (256,307) (2,256) 